In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file_name = "CRM Sales.xlsx"

# Single out the Accounts sheet
df_accounts = pd.read_excel(file_name, sheet_name = "accounts")
df_accounts.head(10)

In [ ]:
df_accounts.shape

In [ ]:
df_accounts.info()

In [ ]:
# Change spelling from "technolgy" to "technology"
df_accounts["sector"] = df_accounts["sector"].replace("technolgy", "technology", regex = True)

Analyze the Distribution of Accounts across Industries and Revenue Segments

In [ ]:
# Industry distribution
df_industry_dist = df_accounts["sector"].value_counts()
df_industry_dist

In [ ]:
# Revenue Segmentation
bins = [0, 10, 50, 100, 500, 1000, 5000, float("inf")] # In millions USD
labels = ["<10M", "10M-50M", "50M-100M", "100M-500M", "500M-1B", "1B-5B", ">5B"]

# Create new column "revenue segmentation"
df_accounts['revenue_segmentation'] = pd.cut(df_accounts["revenue"], bins=bins, labels=labels)
revenue_dist = df_accounts["revenue_segmentation"].value_counts()
revenue_dist

In [ ]:
# Print out Industry and Revenue distributions
print("Industry Distribution:\n", df_industry_dist)
print("\nRevenue Distribution:\n", revenue_dist)

In [ ]:
df_accounts.head()

Identify Top-Performing Products Based on Close Rate

In [ ]:
# Load products sheet
df_products = pd.read_excel(file_name, sheet_name="products")
df_products

In [ ]:
# Load Sales Pipeline sheet
df_sales_pipeline = pd.read_excel(file_name, sheet_name="sales_pipeline")
df_sales_pipeline.head(10)

In [ ]:
# Check details of sales pipeline
df_sales_pipeline.info()

In [ ]:
# Merge sales pipeline with Products on product name
df_pipeline_products = df_sales_pipeline.merge(df_products, on="product", how="left")
df_pipeline_products.head()

In [ ]:
# Calculate close rate per product
close_rate = df_pipeline_products.groupby("product").apply(
    lambda x: (x["deal_stage"] == "Won").sum() / len(x)
).reset_index(name="close_rate")

In [ ]:
# Sort products by close rate
df_top_products = close_rate.sort_values(by="close_rate", ascending=False)
df_top_products

Analyze Performance of Sales Teams, Agents, and Managers across Regions and Product Lines

In [ ]:
# Load sales teams sheet
df_sales_teams = pd.read_excel(file_name, sheet_name="sales_teams")
df_sales_teams.head(10)

In [ ]:
# Check unique regional offices
df_sales_teams["regional_office"].unique()

In [ ]:
# Merge Sales Teams with Sales Pipeline on sales_agent
df_pipeline_teams = df_sales_pipeline.merge(df_sales_teams, on="sales_agent", how="left")
df_pipeline_teams.head()

In [ ]:
# Calculate performance by region, agent and manager
df_team_performance = df_pipeline_teams.groupby(["regional_office", "sales_agent", "manager"]).agg(
    total_deal_value=("close_value", "sum"),
    won_deals=("deal_stage", lambda x: (x == "Won").sum()),
    total_deals=("deal_stage", "count")
)
df_team_performance.head(10)

In [ ]:
# Calculate conversion rate
df_team_performance['conversion_rate'] = df_team_performance["won_deals"] / df_team_performance["total_deals"]
df_team_performance = df_team_performance.reset_index()
df_team_performance.head()

Identify Top-Performing Sales Teams and Individuals

In [ ]:
# Sort by total_deal_value and conversion_rate to identify top performing teams
df_top_teams_by_value = df_team_performance.sort_values(by="total_deal_value", ascending=False)
df_top_individuals_by_conversion = df_team_performance.sort_values(by="conversion_rate", ascending=False)

df_top_teams_by_value.head(10)

In [ ]:
df_top_individuals_by_conversion.head(10)

Identify Patterns or Trends in Deal Values, Close Dates, and Product Preferences across Industries or Account Segments

In [ ]:
# Merging Sales_pipeline, Accounts and Products sheets on account 
df_pipeline_accounts_products = df_sales_pipeline.merge(df_accounts, on="account", how="left").merge(df_products, on="product", how="left")
df_pipeline_accounts_products.head()

In [ ]:
df_pipeline_accounts_products.info()

In [ ]:
# Analysis on deal values over time
df_deal_value_trend = df_pipeline_accounts_products.groupby(df_pipeline_accounts_products['close_date'].dt.to_period("M")).agg(
    total_deal_value=('close_value', 'sum')
).reset_index()
df_deal_value_trend.sort_values(by="total_deal_value", ascending=False)

In [ ]:
# Product preference by industry
df_product_preference_by_sector = df_pipeline_accounts_products.groupby(['sector', 'product']).size().unstack().fillna(0)
df_product_preference_by_sector.head(10)

In [ ]:
# Create an Excel writer object
file_path = 'Analysed CRM Sales.xlsx'
with pd.ExcelWriter(file_path) as writer:
    # Write each DataFrame to a different sheet
    df_accounts.to_excel(writer, sheet_name='Accounts', index=False)
    df_products.to_excel(writer, sheet_name='Products', index=False)
    df_sales_pipeline.to_excel(writer, sheet_name='Sales_Pipeline', index=False)
    df_pipeline_products.to_excel(writer, sheet_name='Sales_Pipeline & Products', index=False)
    df_top_products.to_excel(writer, sheet_name='Top Products', index=False)
    df_sales_teams.to_excel(writer, sheet_name='Sales Teams', index=False)
    df_pipeline_teams.to_excel(writer, sheet_name='Sales Teams & Sales Pipeline', index=False)
    df_team_performance.to_excel(writer, sheet_name='Team Performance', index=False)
    df_top_teams_by_value.to_excel(writer, sheet_name='Top Teams by Value', index=False)
    df_top_individuals_by_conversion.to_excel(writer, sheet_name='Top Individuals by Conversion', index=False)
    df_pipeline_accounts_products.to_excel(writer, sheet_name='Sales Pipeline, Accounts & Products', index=False)
    df_deal_value_trend.to_excel(writer, sheet_name='Deal Value Trend over Time', index=False)
    df_product_preference_by_sector.to_excel(writer, sheet_name='Product Preference by Sector', index=False)